### Install and import the dependecies

In [ ]:
!pip install "langchain==0.0.345" | tail -n 1
!pip install wget | tail -n 1
!pip install sentence-transformers | tail -n 1
!pip install "chromadb==0.3.26" | tail -n 1
!pip install "ibm-watson-machine-learning>=1.0.335" | tail -n 1
!pip install "pydantic==1.10.0" | tail -n 1

In [232]:
import os, getpass

### watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [15]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey":"09naIbU_40U9bzS3yw2ZiRkDY1va1k5FCwei05DVa3k9"
}

### Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

**Hint**: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be `Projects / <project name> /`. Click on the `<project name>` link. Then get the `project_id` from Project's Manage tab (Project -> Manage -> General -> Details).


In [16]:
try:
    project_id = "0ece3ec4-a895-4b50-aca2-5794f884c3bb"
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="data"></a>
## Document data loading

Download the file with State of the Union.

In [456]:
# import wget

# filename = 'BigCash.pdf'
# url = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/foundation_models/state_of_the_union.txt'

# if not os.path.isfile(filename):
#     wget.download(url, out=filename)

<a id="build_base"></a>
## Build up knowledge base

The most common approach in RAG is to create dense vector representations of the knowledge base in order to calculate the semantic similarity to a given user query.

In this basic example, we take the State of the Union speech content (filename), split it into chunks, embed it using an open-source embedding model, load it into <a href="https://www.trychroma.com/" target="_blank" rel="noopener no referrer">Chroma</a>, and then query it.

In [457]:
from langchain.document_loaders import PyPDFDirectoryLoader

In [458]:
# from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
filename="data/BigCash.pdf"
# fileDir="C:/Users/PriKumar/AIML/IBM POC/data"
# loader = TextLoader(filename)
loader = PyPDFLoader(filename) 
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [459]:
texts

[Document(page_content='Big Cash App  \nPRODUCT NOTE', metadata={'source': 'data/BigCash.pdf', 'page': 0}),
 Document(page_content='1 \n TABLE OF CONTENTS  \nCHAPTER 1: REAL MONEY GAMING ................................ ................................ ................................ .............. 6 \nAbout  ................................ ................................ ................................ ................................ ................................ ......6 \nIntroduction to BigCash  ................................ ................................ ................................ ................................ ......... 6 \nGames in BigCash ................................ ................................ ................................ ................................ ..................... 7 \nOnboarding process ................................ ................................ ................................ ................................ ................

The dataset we are using is already split into self-contained passages that can be ingested by Chroma.

### Create an embedding function

Note that you can feed a custom embedding function to be used by chromadb. The performance of Chroma db may differ depending on the embedding model used.

In [460]:
persist_directory="C:/Users/PriKumar/AIML/IBM POC/storage/bgc/"
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings,persist_directory=persist_directory)

In [461]:
docsearch.persist()
# docsearch=None

<a id="models"></a>
## Foundation Models on `watsonx.ai`

IBM watsonx foundation models are among the <a href="https://python.langchain.com/docs/integrations/llms/watsonxllm" target="_blank" rel="noopener no referrer">list of LLM models supported by Langchain</a>. This example shows how to communicate with <a href="https://newsroom.ibm.com/2023-09-28-IBM-Announces-Availability-of-watsonx-Granite-Model-Series,-Client-Protections-for-IBM-watsonx-Models" target="_blank" rel="noopener no referrer">Granite Model Series</a> using <a href="https://python.langchain.com/docs/get_started/introduction" target="_blank" rel="noopener no referrer">Langchain</a>.

### Defining model
You need to specify `model_id` that will be used for inferencing:

In [17]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

# model_id = ModelTypes.GRANITE_13B_CHAT_V2

In [18]:
model_id=ModelTypes.FLAN_UL2

### Defining the model parameters
We need to provide a set of model parameters that will influence the result:

In [19]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

### LangChain CustomLLM wrapper for watsonx model
Initialize the `WatsonxLLM` class from Langchain with defined parameters and `ibm/granite-13b-chat-v2`. 

In [20]:
from langchain.llms import WatsonxLLM

watsonx_granite = WatsonxLLM(
    model_id=model_id.value,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

In [21]:
watsonx_granite

WatsonxLLM(model_id='google/flan-ul2', project_id='0ece3ec4-a895-4b50-aca2-5794f884c3bb', url=SecretStr('**********'), apikey=SecretStr('**********'), params={'decoding_method': <DecodingMethods.GREEDY: 'greedy'>, 'min_new_tokens': 1, 'max_new_tokens': 100, 'stop_sequences': ['<|endoftext|>']}, watsonx_model=<ibm_watson_machine_learning.foundation_models.model.Model object at 0x000001A3073868F0>)

<a id="predict"></a>
## Generate a retrieval-augmented response to a question

Build the `RetrievalQA` (question answering chain) to automate the RAG task.

In [467]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=watsonx_granite, chain_type="stuff", retriever=docsearch.as_retriever())

### Select questions

Get questions from the previously loaded test dataset.

In [491]:
query = "how to redeem money?"
print(qa.run(query))

Open BigCash app Open Menu and tap on Redee m Check your wallet, your balance will be divided into two parts. Redeem Successful Enter and Re -enter Mobile no./UPI id and then tap on the redeem button. Enter the amount to redeem. NOTE:  9% processing fee is applicable if the user redeems below Rs. 500.00  5% processing fee is applicable if the user redeems Rs. 500 or


In [469]:
docsearch = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
qa2 = RetrievalQA.from_chain_type(llm=watsonx_granite, chain_type="stuff", retriever=docsearch.as_retriever())

In [492]:
query = "types of games in big cash?"
print(qa2.run(query))

Card Games Casual Games Fantasy Games Poker Cricket Rummy Football Call Break Kabaddi - Play Off Multiplayer Knife Hit Egg Toss 8 Ball Pool Ice Blaster Bulb Smash Basketball Fruit Chop Car Race Cricket Cash Soccer Big Ludo Candy Cash Fast Ludo


In [229]:
result=qa2.run(query)

In [181]:
result

'Open BigCash app Open Menu and tap on Redee m Check your wallet, your balance will be divided into two parts. Redeem Successful Enter and Re -enter Mobile no./UPI id and then tap on the redeem button. Enter the amount to redeem.'

In [503]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey":"yGZifqC6xI5S-7VgMxjs2XLXuidVMJMulVaW-e-3JTzX"
}

try:
    project_id = "7ca912bc-4d80-4814-9b13-0706a7acd3e3"
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

# from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
fileDir="C:/Users/PriKumar/AIML/IBM POC/data/"
# loader = TextLoader(filename)
# filename2='data/NEWFAQs.pdf'
loader2 = PyPDFDirectoryLoader(fileDir)
documents2 = loader2.load()
text_splitter2 = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts2 = text_splitter2.split_documents(documents2)


In [504]:
texts2

[Document(page_content='Big Cash App  \nPRODUCT NOTE', metadata={'source': 'C:\\Users\\PriKumar\\AIML\\IBM POC\\data\\BigCash.pdf', 'page': 0}),
 Document(page_content='1 \n TABLE OF CONTENTS  \nCHAPTER 1: REAL MONEY GAMING ................................ ................................ ................................ .............. 6 \nAbout  ................................ ................................ ................................ ................................ ................................ ......6 \nIntroduction to BigCash  ................................ ................................ ................................ ................................ ......... 6 \nGames in BigCash ................................ ................................ ................................ ................................ ..................... 7 \nOnboarding process ................................ ................................ ................................ ............

In [505]:
# persist_directory='C:/Users/PriKumar/AIML/IBM POC'
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
persist_directory2="C:/Users/PriKumar/AIML/IBM POC/storage/faq"
# embeddings = HuggingFaceEmbeddings()
docsearchFAQ = Chroma.from_documents(texts2, embeddings,persist_directory=persist_directory2)
docsearchFAQ.persist()

In [523]:
vectordbFAQ_BGC = Chroma(persist_directory="C:/Users/PriKumar/AIML/IBM POC/storage/faq", embedding_function=embeddings)
qa_chain = RetrievalQA.from_chain_type(llm=watsonx_granite, chain_type="stuff", retriever=vectordbFAQ_BGC.as_retriever())

In [521]:
# model_id2 = ModelTypes.GRANITE_13B_CHAT_V2
# chat = WatsonxLLM(
# model_id=model_id2.value,
# url=credentials.get("url"),
# apikey=credentials.get("apikey"),
# project_id=project_id,
# params=parameters)
# qa2 = RetrievalQA.from_chain_type(llm=chat, chain_type="stuff", retriever=docsearch.as_retriever())
query="what is token wallet?"

result=qa_chain.run(query)
print(result)
# if result=='Unhelpful' or "I don't know":
#     result2=qa2.run(query)
#     print(result2)

Token is virtual currency which can be purchased by the user to play token games on the platform.


Failure during generate. (POST https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text?version=2023-12-05)
Status code: 403, body: {"errors":[{"code":"no_associated_service_instance_error","message":"WML instance 9c4e7aaf-115a-4515-a78c-65841de345a3 status is not active, current status: Inactive"}],"trace":"e0e653a5d07b068181ea41bbdf8cd063","status_code":403}


ApiRequestFailure: Failure during generate. (POST https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text?version=2023-12-05)
Status code: 403, body: {"errors":[{"code":"no_associated_service_instance_error","message":"WML instance 9c4e7aaf-115a-4515-a78c-65841de345a3 status is not active, current status: Inactive"}],"trace":"e0e653a5d07b068181ea41bbdf8cd063","status_code":403}

In [4]:
# from langchain.chains import RetrievalQA

# # qa2 = VectorDBQA.from_chain_type(llm=watsonx_granite, chain_type="stuff", retriever=docsearch.as_retriever())
# pdf_qa = ConversationalRetrievalChain.from_llm(watsonx_granite, docsearch.as_retriever(search_kwargs={'k': 8}), verbose=False)

# yellow = "\033[0;33m"
# green = "\033[0;32m"
# white = "\033[0;39m"

# chat_history = []
# while True:
#     query = input(f"{green}Prompt: ")
#     if query == "exit" or query == "quit" or query == "q" or query == "f":
#         print('Exiting')
#         sys.exit()
#     if query == '':
#         continue
#     result = pdf_qa(
#         {"question": query, "chat_history": chat_history})
#     print(f"{white}Answer: " + result["answer"])
#     chat_history.append((query, result["answer"]))

In [54]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey":"09naIbU_40U9bzS3yw2ZiRkDY1va1k5FCwei05DVa3k9"
}

try:
    project_id = "0ece3ec4-a895-4b50-aca2-5794f884c3bb"
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

# from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
fileDir="C:/Users/PriKumar/AIML/IBM POC/data/"
# loader = TextLoader(filename)
# filename2='data/NEWFAQs.pdf'
loader3 = PyPDFDirectoryLoader(fileDir)
documents3 = loader3.load()
text_splitter3 = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0)
texts3 = text_splitter3.split_documents(documents3)


In [55]:
texts3

[Document(page_content='Big Cash App  \nPRODUCT NOTE', metadata={'source': 'C:\\Users\\PriKumar\\AIML\\IBM POC\\data\\BigCash.pdf', 'page': 0}),
 Document(page_content='1 \n TABLE OF CONTENTS  \nCHAPTER 1: REAL MONEY GAMING ................................ ................................ ................................ .............. 6 \nAbout  ................................ ................................ ................................ ................................ ................................ ......6', metadata={'source': 'C:\\Users\\PriKumar\\AIML\\IBM POC\\data\\BigCash.pdf', 'page': 1}),
 Document(page_content='Introduction to BigCash  ................................ ................................ ................................ ................................ ......... 6 \nGames in BigCash ................................ ................................ ................................ ................................ ..................... 7', metadata={'sourc

In [86]:
# persist_directory='C:/Users/PriKumar/AIML/IBM POC'
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
persist_directory3="C:/Users/PriKumar/AIML/IBM POC/storage/rc_docs"
embeddings = HuggingFaceEmbeddings()
docsearch= Chroma.from_documents(texts3, embeddings,persist_directory=persist_directory3)
docsearch.persist()

In [87]:
from langchain.chains import RetrievalQA
vectordb = Chroma(persist_directory="C:/Users/PriKumar/AIML/IBM POC/storage/rc_docs", embedding_function=embeddings)

qa_chain = RetrievalQA.from_chain_type(llm=watsonx_granite, chain_type="stuff", retriever=vectordb.as_retriever())


In [103]:
qa_chain = RetrievalQA.from_chain_type(llm=watsonx_granite, chain_type="map_redu", retriever=vectordb.as_retriever())

In [105]:
qa_chain.run('Multiple accounts can be made or not?')

C:\Users\PriKumar\AppData\Roaming\Python\Python310\site-packages\langchain\chains\llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


ValueError: Could not parse output: 100